source API URL : "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

JSON Target File Path : "abfss://working-labs@datalakestorageaccountname.dfs.core.windows.net/bronze/geo-location/

In [0]:
geoLocationSourceAPIBaseURL = "https://geocoding-api.open-meteo.com/v1/search?name="
geoLocationSourceAPIURLOptions = "&count=10&language=en"

geoLocationSinkLayerName = "bronze"
geoLocationSinkStorageAccountName = "adlsudadalakehousedev"
geoLocationSinkFolderName = "geo-location"

geoLocationSinkFolderPath = f"abfss://{geoLocationSinkLayerName}@{geoLocationSinkStorageAccountName}.dfs.core.windows.net/{geoLocationSinkFolderName}"

In [0]:
import requests
import json 
import pandas as pd

In [0]:
geoLocationAPIResponse = requests.get(geoLocationSourceAPIURL).json()
geoLocationPandasDF = pd.DataFrame(geoLocationAPIResponse["results"])
geoLocationSparkDF = spark.createDataFrame(geoLocationPandasDF) 

In [0]:
display(geoLocationSparkDF)

In [0]:
marketNamesDF = spark.sql("select MARKET_NAME from  pricing_analytics.gold.reporting_dim_market_gold")

In [0]:
marketNamesList = [market_name["MARKET_NAME"] for market_name in marketNamesDF.collect()]

In [0]:
geoLocationAPIResponse =[]

for marketName in marketNamesList:
    markeNameUrl = geoLocationSourceAPIBaseURL+ marketName +geoLocationSourceAPIURLOptions 
    response = requests.get(markeNameUrl).json()
    if 'results' in response  : 
        geoLocationAPIResponse.append(response)
geoLocationPandasDF = pd.DataFrame(geoLocationAPIResponse)
geoLocationSparkDF = spark.createDataFrame(geoLocationPandasDF) 
(geoLocationSparkDF
.write
.mode("overwrite")
.json(geoLocationSinkFolderPath)
)


In [0]:
geoLocationPandasDF

In [0]:
display(geoLocationSparkDF)